#Run this before starting
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"


In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import torch.optim as optim
from torch_geometric.nn.conv import feast_conv


In [35]:
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')

print("Using {}.".format(device_name))

Using cpu.


In [48]:
#data

import numpy as np
import torch
import random
from torch_geometric.data import Data, DataLoader

#Import data function
def get_data(filepath):
    data = np.loadtxt(filepath)
    x = data[:, :20]
    z = data[:, 20:40]
    m = data[:, 40:60]

    dataX = []
    for i in range(x.shape[0]):
        dataXs = np.column_stack((np.arange(20), x[i], z[i]))
        dataX.append(dataXs)

    dataY = m
    zipped = list(zip(dataX, dataY))
    np.random.shuffle(zipped)
    dataX, dataY = zip(*zipped)

    dataX = np.array(dataX)
    dataY = np.array(dataY)

    dataEdgeIndex = np.column_stack((np.arange(20)[:-1], np.arange(20)[1:]))
    dataEdgeIndex = np.vstack((dataEdgeIndex, dataEdgeIndex[:, ::-1])).T

    dataset = [Data(x=torch.from_numpy(x).float(), edge_index=torch.from_numpy(dataEdgeIndex).long(), y=torch.from_numpy(y).float()) for x, y in zip(dataX, dataY)]

    return dataset


#Getting data
dataset = get_data("data1.txt")
print(len(dataset))
#Splitting dataset
datasetRun = dataset[:int(len(dataset)*0.90)]
datasetTest = dataset[int(len(dataset)*0.90):]

train_loader = datasetRun[:int(len(datasetRun)*0.75)]
test_loader = datasetRun[int(len(datasetRun)*0.75):]
"""
train_loader = train_loader[:100]
test_loader = test_loader[:25]
"""

#Checks

print('train_loader size: ', len(train_loader))
print('test_loader size: ', len(test_loader))


# one data object from train_loader:
graph = train_loader[0]
print(graph.x.shape)
print(graph.edge_index.shape)
print(graph.y.shape)

7678
train_loader size:  5182
test_loader size:  1728
torch.Size([20, 3])
torch.Size([2, 38])
torch.Size([20])


In [49]:
class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True):
        super(ArchNN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        #self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        #self.conv5 = feast_conv.FeaStConv(256, 512, heads=heads, t_inv=t_inv)
        #self.conv6 = feast_conv.FeaStConv(512, 256, heads=heads, t_inv=t_inv)
        #self.conv7 = feast_conv.FeaStConv(256, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        #self.conv4.reset_parameters()
        #self.conv5.reset_parameters()
        #self.conv6.reset_parameters()
        #self.conv7.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.fc0(x))
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = F.elu(self.conv3(x, edge_index))
        #x = F.elu(self.conv4(x, edge_index))
        #x = F.elu(self.conv5(x, edge_index))
        #x = F.elu(self.conv6(x, edge_index))
        #x = F.elu(self.conv7(x, edge_index))
        x = F.elu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #F.log_softmax(x, dijm=1)
        x = torch.squeeze(x, dim=1)
        return x




In [50]:
import time
import torch
import torch.nn.functional as F


def print_info(info):
    message = ('Epoch: {}/{}, Duration: {:.3f}s,'
               'Train Loss: {:.4f}, Test Loss:{:.4f}').format(
                   info['current_epoch'], info['epochs'], info['t_duration'],
                   info['train_loss'], info['test_loss'])
    print(message)


def run(model, train_loader, test_loader, num_nodes, epochs, optimizer, device):

    for epoch in range(1, epochs + 1):
        t = time.time()
        train_loss = train(model, train_loader, optimizer, device)
        t_duration = time.time() - t
        test_loss = test(model, test_loader, num_nodes, device)
        eval_info = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'current_epoch': epoch,
            'epochs': epochs,
            't_duration': t_duration
        }

        print_info(eval_info)


def train(model, train_loader, optimizer, device):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        losses = F.mse_loss(output, data.y.to(device), reduction='none')  # Calculate MSE loss for each item in output and labels
        loss = losses.mean()  # Compute mean loss for backpropagation
        loss.backward(retain_graph=True)
        #loss = model.compute_loss(output, data.y)  # compute loss with L1 regularization
        #loss = F.nll_loss(log_probs, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(model, test_loader, num_nodes, device):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            out = model(data.to(device))
            total_loss += F.mse_loss(out, data.y.to(device)).item()
            #pred = out.max(1)[1]
            #correct += pred.eq(data.y).sum().item()
            #n_graphs += data.num_graphs
    return total_loss / len(test_loader)

In [53]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

model = ArchNN(num_features, num_nodes, heads=1).to(device_name)

optimizer = optim.Adam(model.parameters(),
                       lr=0.0001,
                       weight_decay=0.001)


run(model, train_loader, test_loader, num_nodes, 100, optimizer, device_name)

torch.save(model, "apr18.pt")

Epoch: 1/100, Duration: 42.933s,Train Loss: 5.5934, Test Loss:4.4755
Epoch: 2/100, Duration: 46.307s,Train Loss: 3.3071, Test Loss:2.8194
Epoch: 3/100, Duration: 56.143s,Train Loss: 2.1562, Test Loss:1.9985
Epoch: 4/100, Duration: 45.732s,Train Loss: 1.5677, Test Loss:1.3895
Epoch: 5/100, Duration: 49.264s,Train Loss: 1.1568, Test Loss:0.9881
Epoch: 6/100, Duration: 46.661s,Train Loss: 0.9285, Test Loss:0.7710
Epoch: 7/100, Duration: 50.323s,Train Loss: 0.7896, Test Loss:0.6327
Epoch: 8/100, Duration: 54.681s,Train Loss: 0.6858, Test Loss:0.5348
Epoch: 9/100, Duration: 48.862s,Train Loss: 0.5986, Test Loss:0.4639
Epoch: 10/100, Duration: 46.966s,Train Loss: 0.5260, Test Loss:0.4053
Epoch: 11/100, Duration: 46.569s,Train Loss: 0.4642, Test Loss:0.3563
Epoch: 12/100, Duration: 47.994s,Train Loss: 0.4117, Test Loss:0.3168
Epoch: 13/100, Duration: 47.407s,Train Loss: 0.3636, Test Loss:0.2801
Epoch: 14/100, Duration: 46.450s,Train Loss: 0.3195, Test Loss:0.2405
Epoch: 15/100, Duration: 46.3

In [52]:
# save model

torch.save(model, "apr18.pt")

In [55]:
#load model

model = torch.load("apr18.pt")
model.eval()
tester = model(datasetTest[0].to(device_name))
print(datasetTest[0].y)
print(tester)
tester = model(datasetTest[1].to(device_name))
print(datasetTest[1].y)
print(tester)
tester = model(datasetTest[2].to(device_name))
print(datasetTest[2].y)
print(tester)

tensor([ 1.5929, -0.0468,  0.8205,  1.1202,  0.9881,  0.5728,  0.0228, -0.5255,
        -0.9589, -1.1967, -1.1967, -0.9589, -0.5255,  0.0228,  0.5728,  0.9881,
         1.1202,  0.8205, -0.0468, -1.5929])
tensor([ 1.6214, -0.0345,  0.8675,  1.1567,  1.0123,  0.5758,  0.0064, -0.5194,
        -0.9393, -1.1778, -1.1911, -0.9689, -0.5195,  0.0392,  0.5983,  1.0137,
         1.0970,  0.9285,  0.1239, -1.1868], grad_fn=<SqueezeBackward1>)
tensor([-0.3996,  0.9465,  1.0823,  0.9144,  0.5465,  0.0754, -0.4131, -0.8463,
        -1.1675, -1.3381, -1.3381, -1.1675, -0.8463, -0.4131,  0.0754,  0.5465,
         0.9144,  1.0823,  0.9465,  0.3996])
tensor([-0.5025,  0.8308,  1.0607,  0.9177,  0.5771,  0.0946, -0.3914, -0.8011,
        -1.1205, -1.3207, -1.3267, -1.1601, -0.8647, -0.4389,  0.0315,  0.5263,
         1.0105,  1.3051,  1.2712,  0.9909], grad_fn=<SqueezeBackward1>)
tensor([-0.9057,  0.9870,  0.8736,  0.6225,  0.2890, -0.0760, -0.4270, -0.7255,
        -0.9416, -1.0549, -1.0549, -0.9416, 

In [25]:
model_tester_data = "testdata.txt"
model_tester_dataset = create_dataset(model_tester_data)

print(model_tester_dataset[0].y)
tester = model(model_tester_dataset[0].to(device_name))

print(tester)

print(model_tester_dataset[1].y)
tester2 = model(model_tester_dataset[1].to(device_name))
print(tester2)

tensor([ 1.3550,  0.1630,  0.9600,  1.1790,  0.9760,  0.5080, -0.0740, -0.6390,
        -1.0780, -1.3180, -1.3180, -1.0780, -0.6390, -0.0740,  0.5080,  0.9760,
         1.1790,  0.9600,  0.1630, -1.3550])
tensor([ 1.4588,  0.0168,  0.9337,  1.0978,  0.8159,  0.3859, -0.1911, -0.7819,
        -1.2410, -1.4660, -1.4306, -1.0969, -0.5489,  0.1082,  0.5618,  0.9636,
         1.3206,  0.9643,  0.0294, -1.8978], grad_fn=<SqueezeBackward1>)
tensor([ 1.6200,  0.1130,  1.0340,  1.3030,  1.0950,  0.5870, -0.0530, -0.6770,
        -1.1640, -1.4290, -1.4290, -1.1640, -0.6770, -0.0530,  0.5870,  1.0950,
         1.3030,  1.0340,  0.1130, -1.6200])
tensor([ 1.7416, -0.0129,  1.0249,  1.2137,  0.9230,  0.4418, -0.1897, -0.8252,
        -1.3337, -1.5737, -1.4708, -1.0195, -0.3363,  0.4624,  0.9845,  1.3977,
         1.5277,  1.0012, -0.6278, -2.5273], grad_fn=<SqueezeBackward1>)


In [38]:
#OLD
# Define a simple GNN model
class MyGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyGNN, self).__init__()
        
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return x

# Define a simple training function for the GNN model
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = F.mse_loss(output, data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Define a simple testing function for the GNN model
def test(model, data):
    model.eval()
    output = model(data.x, data.edge_index)
    return output

# Set up the input data
x = torch.tensor([
    [0,0,1,1,1,0,0,0],
    [10,5,0,0,0,0,5,0],
    [20,10,0,0,0,0,5,0],
    [30,5,0,0,0,0,5,0],
    [40,0,1,1,1,0,0,0]
], dtype=torch.float)

edge_index = torch.tensor([
    [0,1], [1,0], [1,2], [2,1], [2,3], [3,2], [3,4], [4,3]
], dtype=torch.long).t()

# Set up the output data (nodal forces and moments)
y = torch.tensor([
    [0,0,0,0,0,10,20,0],
    [0,0,0,0,0,5,0,0],
    [0,0,0,0,0,5,0,0],
    [0,0,0,0,0,5,0,0],
    [0,0,0,0,0,10,20,0]
], dtype=torch.float)

# Create a Data object that encapsulates the input and output data
data = Data(x=x, edge_index=edge_index, y=y)

# Initialize the GNN model
input_dim = x.shape[1]
output_dim = y.shape[1]
hidden_dim = 16
model = MyGNN(input_dim, hidden_dim, output_dim)

# Define the optimizer and the number of epochs for training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
num_epochs = 100

# Train the GNN model
for epoch in range(num_epochs):
    loss = train(model, data, optimizer)
    print('Epoch {}: loss={}'.format(epoch, loss))

# Test the GNN model on a new input example
x_new = torch.tensor([
    [0,0,1,1,1,0,0,0],
    [10,5,0,0,0,0,5,0],
    [20,10,0,0,0,0,5,0],
    [30,5,0,0,0,0,5,0]
])
print(edge_index)

Epoch 0: loss=27.194686889648438
Epoch 1: loss=23.968759536743164
Epoch 2: loss=21.4787540435791
Epoch 3: loss=19.75748062133789
Epoch 4: loss=18.83110237121582
Epoch 5: loss=18.46010971069336
Epoch 6: loss=18.401479721069336
Epoch 7: loss=18.3200740814209
Epoch 8: loss=18.0425968170166
Epoch 9: loss=17.594493865966797
Epoch 10: loss=17.073143005371094
Epoch 11: loss=16.612995147705078
Epoch 12: loss=16.298181533813477
Epoch 13: loss=16.103126525878906
Epoch 14: loss=16.005006790161133
Epoch 15: loss=15.95953369140625
Epoch 16: loss=15.92503547668457
Epoch 17: loss=15.877996444702148
Epoch 18: loss=15.810765266418457
Epoch 19: loss=15.72706127166748
Epoch 20: loss=15.643503189086914
Epoch 21: loss=15.57574462890625
Epoch 22: loss=15.531913757324219
Epoch 23: loss=15.508995056152344
Epoch 24: loss=15.494709968566895
Epoch 25: loss=15.474119186401367
Epoch 26: loss=15.437162399291992
Epoch 27: loss=15.396413803100586
Epoch 28: loss=15.341238021850586
Epoch 29: loss=15.272732734680176
Epo